In [1]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import glob
from niceplots import geographic
from global_land_mask import globe
from lethal_heat import Vecellio22
import pandas as pd
from niceplots import geographic
from global_land_mask import globe
from matplotlib.colors import LinearSegmentedColormap
import rioxarray

In [2]:
model_wl_dict = {'ACCESS-CM2':   [2015, 2027, 2040, 2049, 2056, 2066], 
                 'ACCESS-ESM1-5': [2017, 2029, 2039, 2052, 2063, 2069],
                 'CNRM-CM6-1':   [2019, 2031, 2045, 2052, 2059, 2066], 
                 'CNRM-ESM2-1':  [2024, 2035, 2047, 2056, 2066, 2074], 
                 'CanESM5':      [1998, 2013, 2026, 2031, 2040, 2049],
                 'EC-Earth3-Veg-LR':[2007, 2031, 2044, 2056, 2064, 2069],
                 'FGOALS-g3':    [2006, 2030, 2047, 2054, 2074, 2087], 
                 'GFDL-CM4':     [2017, 2029, 2044, 2052, 2059, 2072],
                 'INM-CM4-8':    [2016, 2029, 2045, 2059, 2070, 2086], 
                 'INM-CM5-0':    [2019, 2035, 2048, 2065, 2072, 2090], 
                 'IPSL-CM6A-LR': [2008, 2023, 2038, 2043, 2051, 2062],
                 'MIROC-ES2L':   [2023, 2039, 2052, 2061, 2073, 2087], 
                 'MIROC6':       [2024, 2043, 2054, 2069, 2079, 2089], 
                 'MPI-ESM1-2-HR':[2016, 2034, 2053, 2063, 2076, 2087],
                 'MPI-ESM1-2-LR':[2021, 2037, 2052, 2061, 2074, 2083], 
                 'MRI-ESM2-0':   [2015, 2027, 2037, 2055, 2065, 2073]}
levels = [1, 1.5, 2, 2.5, 3, 3.5,]
window_size = 10
model_names = list(model_wl_dict)
n_models = len(model_names)
n_wl = len(levels)

### Convert for each model individually

In [ ]:
fp_temp = '/home/davidbyrne/disks/ssd2/projects/carter2/data/tolh_analysis/tolh_ndays6_{0}_ssp585.nc'
fp_out_temp_wl = '/home/davidbyrne/disks/ssd2/projects/carter2/data/tolh_warming_levels/tolh_ndays{0}_wl_{1}.nc'
ndays = 6

for ii, model in enumerate(model_names):
    print(ii+1, n_models, end='\r')
    fp_ii = fp_temp.format(model)
    ds_ii = xr.open_dataset(fp_ii).load()
    n_lon = ds_ii.dims['lon']
    n_lat = ds_ii.dims['lat']
    wl_means = np.zeros((n_wl, n_lat, n_lon))
    
    base_year = pd.to_datetime(ds_ii.time[0].values).year
    
    for jj, wl_jj in enumerate(levels):
        
        wlyear = model_wl_dict[model][jj]
        idx_ctr = wlyear - base_year
        ds_wl = ds_ii.isel(time = slice(idx_ctr - window_size, idx_ctr + window_size))
        ds_wl_mean = ds_wl.mean(dim='time', skipna=True)
        wl_means[jj] = ds_wl_mean[f'ndays_{ndays}'].values
        
    ds_out = ds_ii[['lon','lat']]
    ds_out['level'] = (['level'], levels)
    ds_out['tolh_wlmean'] = (['level','lat','lon'], wl_means)
    fp_out_wl = fp_out_temp.format(ndays, model)
    ds_out.to_netcdf(fp_out_wl)

### Calculate model ensemble median number of days per year.

In [3]:
# Read data from days per year per warming level file.
fp_list = glob.glob('/home/davidbyrne/disks/ssd2/projects/carter2/data/tolh_warming_levels/*ndays6*')
ds_list = [xr.open_dataset(fp, chunks={}) for fp in fp_list]
ds = xr.concat(ds_list, dim='model')
fp_out = '/home/davidbyrne/disks/ssd2/projects/carter2/median_ndays_per_year_6H.tif'

In [4]:
# Calculate model median and standard deviation
ds_med = ds.median(dim='model').compute()['tolh_wlmean']
ds_std = ds.std(dim='model').compute()['tolh_wlmean']
ds_max = ds.max(dim='model').compute()['tolh_wlmean']
ds_min = ds.min(dim='model').compute()['tolh_wlmean']
med = ds_med.values
std = ds_std.values

# Mask out points where the median is zero
std[med == 0] = np.nan
med[med == 0] = np.nan

# Calculate landmask
lon2, lat2 = np.meshgrid(ds_med.lon, ds_med.lat)
landmask = globe.is_land(lat2, lon2)
landmaskW = np.where(~landmask)

In [ ]:
ds_out = ds_med.to_dataset()[['lon','lat']]
ds_out['Band'] = (['level','lat','lon'], med)
ds_out = ds_out.rename({'lon':'x','lat':'y'})
ds_out.attrs['crs'] = 'WGS84'

In [ ]:
ds_out['Band'].rio.to_raster(fp_out)

### 1D Data Analysis (Area etc)

In [ ]:
# Calculate number of grid points with over N days per year
N_days_per_year = 1

# Points that satisfy the criteria -- sum over lon lat (preserving each model for now)
ds_area = ds.tolh_wlmean > N_days_per_year
ds_area = ds_area.sum(dim=('lon','lat'))

# Now get the average and standard deviation over the model ensemble.
ds_area_mean = ds_area.mean(dim='model')
ds_area_std = ds_area.std(dim='model')

In [ ]:
# Calculate total number of days
ds_total = ds.tolh_wlmean.sum(dim=['lon','lat']) / 25920
ds_total_mean = ds_total.mean(dim='model')
ds_total_std = ds_total.std(dim='model')

### First warming level of LH occurrence for different numbers of days

In [5]:
fp_out_gt1 = '/home/davidbyrne/disks/ssd2/projects/carter2/paper_data/firstWL_gt1dayperyear.tif'
fp_out_gt7 = '/home/davidbyrne/disks/ssd2/projects/carter2/paper_data/firstWL_gt7dayperyear.tif'

# 1 day a year
# We concatenate a true layer onto our arrays in case no warming levels were found (then that concat layer will be default)
ds_med_gt1 = np.concatenate( [(ds_med>1).values, np.ones((1, 360, 720))], axis=0 )
ds_med_gt1_amax = np.argmax(ds_med_gt1,axis=0).astype(float) / 2 + 1
ds_med_gt1_amax[ landmaskW[0], landmaskW[1] ] = np.nan

ds_max_gt1 = np.concatenate( [(ds_max>1).values, np.ones((1, 360, 720))], axis=0 )
ds_max_gt1_amax = np.argmax(ds_max_gt1,axis=0).astype(float) / 2 + 1
ds_max_gt1_amax[ landmaskW[0], landmaskW[1] ] = np.nan

ds_min_gt1 = np.concatenate( [(ds_min>1).values, np.ones((1, 360, 720))], axis=0 )
ds_min_gt1_amax = np.argmax(ds_min_gt1,axis=0).astype(float) / 2 + 1
ds_min_gt1_amax[ landmaskW[0], landmaskW[1] ] = np.nan

n_r, n_c = ds_med_gt1_amax.shape
gt1_out = np.zeros((3, n_r, n_c))
gt1_out[0,:,:] = ds_min_gt1_amax
gt1_out[1,:,:] = ds_med_gt1_amax
gt1_out[2,:,:] = ds_max_gt1_amax

# 7 days a year
ds_med_gt7 = np.concatenate( [(ds_med>7).values, np.ones((1, 360, 720))], axis=0 )
ds_med_gt7_amax = np.argmax(ds_med_gt7,axis=0).astype(float) / 2 + 1
ds_med_gt7_amax[ landmaskW[0], landmaskW[1] ] = np.nan

ds_max_gt7 = np.concatenate( [(ds_max>7).values, np.ones((1, 360, 720))], axis=0 )
ds_max_gt7_amax = np.argmax(ds_max_gt7,axis=0).astype(float) / 2 + 1
ds_max_gt7_amax[ landmaskW[0], landmaskW[1] ] = np.nan

ds_min_gt7 = np.concatenate( [(ds_min>7).values, np.ones((1, 360, 720))], axis=0 )
ds_min_gt7_amax = np.argmax(ds_min_gt7,axis=0).astype(float) / 2 + 1
ds_min_gt7_amax[ landmaskW[0], landmaskW[1] ] = np.nan

n_r, n_c = ds_med_gt1_amax.shape
gt7_out = np.zeros((3, n_r, n_c))
gt7_out[0,:,:] = ds_min_gt7_amax
gt7_out[1,:,:] = ds_med_gt7_amax
gt7_out[2,:,:] = ds_max_gt7_amax

In [6]:
ds_out1 = ds_med.to_dataset()[['lon','lat']]
ds_out7 = ds_med.to_dataset()[['lon','lat']]

ds_out1['band_data'] = (['Band','lat','lon'], gt1_out)
ds_out7['band_data'] = (['Band','lat','lon'], gt7_out)

ds_out1.attrs['crs'] = 'WGS84'
ds_out7.attrs['crs'] = 'WGS84'

ds_out1 = ds_out1.rename({'lon':'x','lat':'y'})
ds_out7 = ds_out7.rename({'lon':'x','lat':'y'})

ds_out1['band_data'].rio.to_raster(fp_out_gt1)
ds_out7['band_data'].rio.to_raster(fp_out_gt7)

### Return Periods

In [57]:
fp_temp = '/home/davidbyrne/disks/ssd2/projects/carter2/data/tolh_analysis/tolh_ndays6_{0}_ssp585.nc'
fp_out_rp = '/home/davidbyrne/disks/ssd2/projects/carter2/paper_data/returnperiod_gt1dayperyear.tif'
lon2, lat2 = np.meshgrid(ds_med.lon, ds_med.lat)
landmask = globe.is_land(lat2, lon2)
landmaskW = np.where(~landmask)

In [45]:
data_list = []
for ii, model in enumerate(model_names):
    print(ii+1, n_models, end='\r')
    fp_ii = fp_temp.format(model)
    ds_ii = xr.open_dataset(fp_ii).load()
    n_lon = ds_ii.dims['lon']
    n_lat = ds_ii.dims['lat']
    wl_means = np.zeros((n_wl, n_lat, n_lon))
    data_list.append([])
    
    base_year = pd.to_datetime(ds_ii.time[0].values).year
    
    for jj, wl_jj in enumerate(levels):
        wlyear = model_wl_dict[model][jj]
        idx_ctr = wlyear - base_year
        data_list[ii].append( ds_ii.isel(time = slice(idx_ctr - window_size, idx_ctr + window_size))['ndays_6'].load() )

In [47]:
return_levels = [1]
n_rl = len(return_levels)
return_periods = np.zeros((n_rl, 6, 360, 720))

# Loop over warming levels and calculate return periods
for rr, rl in enumerate(return_levels):
    print(rr, end='\r')
    for ii in range(6):
        data_ii = [data_list[kk][ii] for kk in range(16)]
        data_ii = [tmpds.drop('time') for tmpds in data_ii]
        data_ii_concat = xr.concat(data_ii, dim='model')
        data_ii_concat = data_ii_concat.stack(newdim = ('time','model')) >= rl
        frequency_ii = data_ii_concat.sum('newdim') / data_ii_concat.sizes['newdim']
        return_periods[rr, ii] = 1/frequency_ii
        
return_periods[:,:,landmaskW[0], landmaskW[1]] = np.nan

In [53]:
ds_out = ds_med.to_dataset()[['lon','lat']]
ds_out['return_period'] = (['level','lat','lon'], return_periods[0])
ds_out = ds_out.rename({'lon':'x','lat':'y'})

In [59]:
ds_out['return_period'].rio.to_raster(fp_out_rp)